In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt

import time
from datetime import datetime, timedelta

In [ ]:
sess = None

def reset_vars():
    sess.run(tf.global_variables_initializer())

def reset_tf():
    global sess
    if sess:
        sess.close()
    tf.reset_default_graph()
    sess = tf.Session()

# Convolutional Neural Networks: Solution


### 1) Reset session

In [ ]:
reset_tf()

### 2) Load the data and get the class number

In [ ]:
# Load data
data = input_data.read_data_sets('/tmp/data/', one_hot=True)

# Get class (number) for test data
data.test.cls = np.argmax(data.test.labels, axis=1)

### 3) Set path to where summary files will live

In [ ]:
now = datetime.now()
logs_path = now.strftime("%Y%m%d-%H%M%S") + '/summaries'

### 4) Set parameters

In [ ]:
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

n_classes = 10
n_channels = 1
filt_size = [5, 5]

batch_size = 50
num_iterations = 400 # 1500 # Increase when running
display_step = 100

dropout = 0.5

### 5) Placeholder variables and graph input

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
y_true = tf.placeholder(tf.float32, shape=[None, n_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)
keep_prob = tf.placeholder(tf.float32)

### 6) NN layers

In [ ]:
# Create network of layers
def conv_net(x, img_size, n_classes, stride, filt_size, out_sizes, dropout=None):
    
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, img_size, img_size, 1])

    for out_size in out_sizes[:-1]:
        x = tf.layers.conv2d(x, out_size, filt_size, padding='same', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, (2, 2), (2, 2))
        
    #Fully connected layer
    x = tf.reshape(x, [-1, x.shape[1:].num_elements()])
    x = tf.layers.dense(x, out_sizes[-1], activation=tf.nn.relu)
 
    # Apply Dropout
    if dropout is not None:
        x = tf.nn.dropout(x, dropout)

    # Output, class prediction
    y = tf.layers.dense(x, n_classes, activation=None)
    return y

stride = 1 
out_sizes = [32, 64, 1024]

# for MNIST images
y_pred = conv_net(x, img_size, n_classes, stride, filt_size, out_sizes, dropout=keep_prob)

### 7) Loss

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
tf.summary.scalar('loss', loss)

### 8) Optimizer

In [ ]:
optimizer = tf.train.AdamOptimizer().minimize(loss)

### 9) Accuracy

In [ ]:
y_pred_cls = tf.argmax(y_pred, dimension=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy',accuracy)

### 10) Merge summaries, initialize variables, and launch graph

In [ ]:
#Merge all summaries
merged = tf.summary.merge_all()

#Initialize
reset_vars()

#Create summary writers
train_writer = tf.summary.FileWriter(logs_path + '/train', graph=tf.get_default_graph())
test_writer = tf.summary.FileWriter(logs_path + '/test', graph=tf.get_default_graph())

### 11) Train and test model

In [ ]:
def optimize(num_iterations):
    # Start-time used for printing time-usage below.
    start_time = time.time()

    step = 1
    for i in range(num_iterations):
        
        # Get a batch of training examples.
        x_batch, y_true_batch = data.train.next_batch(batch_size)

        # ---------------------- TRAIN -------------------------
        # optimize model
        sess.run(optimizer, feed_dict={x: x_batch,y_true: y_true_batch, keep_prob: dropout}) 
        
        
        # Print status every 100 iterations.
        if (i % display_step == 0) or (i == num_iterations - 1):
            
            summary = sess.run(merged, feed_dict={x: x_batch, y_true: y_true_batch,  keep_prob: 1.0})
            train_writer.add_summary(summary, step)
            
            #----------------------- TEST ---------------------------
            # test model
            summary, l, acc = sess.run([merged, loss, accuracy], feed_dict={x: data.test.images,
                                                                            y_true: data.test.labels,
                                                                            keep_prob: 1.0})                                                          
            test_writer.add_summary(summary, step)
            
            # Message for network evaluation
            msg = "Optimization Iteration: {0:>6}, Test Loss: {1:>6}, Test Accuracy: {2:>6.1%}"
            print(msg.format(i, l, acc))
            
            step += 1

    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [ ]:
optimize(num_iterations)

### 12) Prediction

In [ ]:
prediction = tf.argmax(y_pred, 1)

def predict(idx):
    image = data.test.images[idx]
    return sess.run(prediction, feed_dict={x: [image], keep_prob: 1.})

idx = 0
actual = np.argmax(data.test.labels[idx])
print ("Predicted: %d, Actual: %d" % (predict(idx), actual))
plt.imshow(data.test.images[idx].reshape((28,28)), cmap=plt.cm.gray_r)

### 13) Close summary writers

In [ ]:
train_writer.close()
test_writer.close()

### 14) Open TensorBoard

In [ ]:
print("Run the command line:\n" \
      "--> tensorboard --logdir=%s " \
      "\nThen open http://0.0.0.0:6006/ into your web browser" %logs_path)

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*